# Clustering Crypto

In [108]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [109]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [110]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [135]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path)
df = df.set_index('Unnamed: 0')
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [136]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [137]:
# Keep only cryptocurrencies that are trading
df = df[(df['IsTrading']==True)]
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [138]:
# Keep only cryptocurrencies with a working algorithm
## Assuming that a 'NaN' in the 'TotalCoinsMined' column is indicative of a broken algorithm
df = df.dropna()
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [139]:
# Remove the "IsTrading" column
df = df.drop(columns='IsTrading',axis=1)
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [140]:
# Remove rows with at least 1 null value
df = df.dropna()
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [141]:
# Remove rows with cryptocurrencies having no coins mined
df = df[(df[['TotalCoinsMined']] != 0).all(axis=1)]
df.tail()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [142]:
# Drop rows where there are 'N/A' text values
df = df[~df['CoinName'].str.contains('N/A')]
df = df[~df['Algorithm'].str.contains('N/A')]
df = df[~df['ProofType'].str.contains('N/A')]
df.tail()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [143]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = df[['CoinName']]
coin_names.head()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [144]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = df.drop(columns=['CoinName'],axis=1)

crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [145]:
# Read 'TotalCoinSupply' as a float rather than an object for subsequent encoding
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype(float)

In [146]:
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [147]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
# Standardize data
## Initialize standard scaler
scaler = StandardScaler()
X_scaler = scaler.fit(X)

## Scale the X data
X_scaled = X_scaler.transform(X)

### Reducing Dimensions Using PCA

In [154]:
# Use PCA to reduce dimensions to 3 principal components
## Initialize the PCA model with 3 principal components
pca = PCA(n_components=3)

# Get THREE principal components for the data.
X_pca = pca.fit_transform(X_scaled)


In [155]:
# Create a DataFrame with the principal components data
df_X_pca = pd.DataFrame(
    data=X_pca, columns=["principal component 1", "principal component 2",'principal component 3']
)
# Set Index to the Index above
df_X_pca = df_X_pca.set_index(crypto_df.index)
df_X_pca.head()


,principal component 1,principal component 2,principal component 3
Unnamed: 0,,,
42,-0.351305,0.921759,-0.573161
404,-0.334457,0.922313,-0.573511
1337,2.330595,1.705466,-0.698367
BTC,-0.136006,-1.292133,0.145351
ETH,-0.141755,-2.001534,0.384832


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [156]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_X_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [163]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_X_pca)
# Predict clusters
predictions = model.predict(df_X_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_df.join(df_X_pca)
clustered_df['CoinName'] = coin_names['CoinName']
clustered_df['Class'] = predictions
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.351305,0.921759,-0.573161,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.334457,0.922313,-0.573511,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.330595,1.705466,-0.698367,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.136006,-1.292133,0.145351,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.141755,-2.001534,0.384832,Ethereum,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [188]:
# Create a 3D-Scatter with the PCA data and the clusters
px.scatter_3d(
    clustered_df,
    x="CoinName",
    y="Algorithm",
    z="ProofType",
    color="Class",
    symbol="Class")

#fig.update_layout(legend=dict(x=0, y=1))
#fig.show()

#### Table of Tradable Cryptocurrencies

In [175]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"],
               sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [176]:
# Print the total number of tradable cryptocurrencies
## Count the number of unique values in 'CoinName' column
tradable_crypto = clustered_df['CoinName'].nunique()
print(f'The total number of tradable cryptocurrencies is, currently, {tradable_crypto}.')

The total number of tradable cryptocurrencies is, currently, 532.


#### Scatter Plot with Tradable Cryptocurrencies

In [178]:
# Scale data to create the scatter plot
## Data has already been scaled

In [185]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(x='TotalCoinsMined',y='TotalCoinSupply',by='CoinName',hover_cols=["CoinName"])



:NdOverlay   [CoinName]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)